# Prevendo Nova Taxa De Inflação com Séries Temporais 

### Inteligência Artificial em Macroeconomia Para Prever a Taxa de Inflação

Como solução para este Lab usaremos um modelo de IA com arquitetura LSTM (Long Short-Term Memory) - LSTM é a melhor arquitetura para criação de modelos de IA que utilizam series temporais

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf # modelo de deep learn
from sklearn.preprocessing import MinMaxScaler # normalizar os dados
from sklearn.metrics import mean_squared_error # calcular o erro
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Carregamento dos dados
dados = pd.read_csv("dataset.csv")

In [5]:
dados.head()

,ano,taxa_juros,taxa_desemprego,PIB,inflacao
0,1970,11.43,5.63,1980.10,0.15
1,1971,14.59,7.34,1930.63,0.39
2,1972,12.45,6.58,1057.70,2.46
3,1973,11.35,14.45,2881.87,0.73
4,1974,9.05,8.05,1617.81,1.48


In [6]:
dados.columns

Index(['ano', 'taxa_juros', 'taxa_desemprego', 'PIB', 'inflacao'], dtype='object')

In [7]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ano              54 non-null     int64  
 1   taxa_juros       54 non-null     float64
 2   taxa_desemprego  54 non-null     float64
 3   PIB              54 non-null     float64
 4   inflacao         54 non-null     float64
dtypes: float64(4), int64(1)
memory usage: 2.2 KB


In [8]:
dados.shape

(54, 5)

In [10]:
# Definindo nossa série temporal

inflacao = dados['inflacao'].values.reshape(-1, 1)

In [11]:
inflacao

array([[ 0.15],
       [ 0.39],
       [ 2.46],
       [ 0.73],
       [ 1.48],
       [ 1.12],
       [ 0.41],
       [ 0.48],
       [ 1.15],
       [ 0.31],
       [ 0.58],
       [ 0.23],
       [ 0.57],
       [ 2.79],
       [ 0.43],
       [ 2.9 ],
       [-0.2 ],
       [ 1.8 ],
       [ 0.  ],
       [ 0.3 ],
       [ 1.72],
       [ 1.1 ],
       [ 2.26],
       [ 0.86],
       [ 2.6 ],
       [-0.17],
       [-0.49],
       [ 2.6 ],
       [ 0.38],
       [-1.  ],
       [ 2.19],
       [ 1.84],
       [ 0.33],
       [-0.38],
       [-2.4 ],
       [ 2.16],
       [ 2.77],
       [-2.45],
       [ 0.82],
       [ 0.64],
       [ 1.3 ],
       [ 0.34],
       [ 0.93],
       [ 0.53],
       [ 0.85],
       [ 0.5 ],
       [ 0.66],
       [ 0.98],
       [ 0.97],
       [ 1.63],
       [ 1.39],
       [ 1.4 ],
       [ 1.17],
       [ 1.27]])

In [12]:
# Função para criar o dataset para o modelo LSTM com look_back
# O parâmetro look back define o numero de passos no tempo que o modelo usará para fazer cada previsõa
# a ideia de "veja os ultimos 3 dados e preveja o 4"

def func_cria_dataset(data, look_back = 1):
   
   # Inicializa duas listas vazias, X e Y, que conterão as sequências de entrada e saída
   X, Y = [], []

   # Itera pelo conjunto de dados menos o valor de look_back, Isso é feiro para evitar índices fora dos limites do array
   for i in range(len(data) - look_back):
      
      # Coleta uma sequência de dados de tamanho look_back começando no índice i
      a = data[i:(i + look_back), 0]

      # Adiciona a sequência à lista X
      X.append(a)

      # Adiciona o valor imediatamente após a sequência de look_back à lista Y. Esse valor será o target.
      Y.append(data[i + look_back, 0])

   # Converte X e Y para arrays numpy para compatibilidade com a maioria das bibliotecas de machine learninog
   return np.array(X), np.array(Y)

In [13]:
# Dividimos os dados em treinamento e teste (respeitando a ordem cronológica dos dados)

indice = int(len(inflacao) * 0.8)
dados_treino, dados_teste = inflacao[0:indice, :], inflacao[indice:len(inflacao), :] 

In [16]:
len(dados_treino), len(dados_teste)

(43, 11)

In [17]:
# Normaliza os dados (Requerimento para redes neurais)

scaler = MinMaxScaler(feature_range = (0, 1))

In [20]:
# Treina e aplica o sclaer emt reino e somente aplica em teste

dados_treino_norm = scaler.fit_transform(dados_treino)
daods_teste_norm = scaler.transform(dados_teste)

In [32]:
#  Criamos os datasets para o modelo LSTM

look_back = 3
X_treino, y_treino = func_cria_dataset(dados_treino_norm, look_back)
X_teste, y_teste = func_cria_dataset(daods_teste_norm, look_back)

In [33]:
X_treino

array([[0.48598131, 0.53084112, 0.91775701],
       [0.53084112, 0.91775701, 0.59439252],
       [0.91775701, 0.59439252, 0.73457944],
       [0.59439252, 0.73457944, 0.66728972],
       [0.73457944, 0.66728972, 0.53457944],
       [0.66728972, 0.53457944, 0.54766355],
       [0.53457944, 0.54766355, 0.6728972 ],
       [0.54766355, 0.6728972 , 0.51588785],
       [0.6728972 , 0.51588785, 0.56635514],
       [0.51588785, 0.56635514, 0.50093458],
       [0.56635514, 0.50093458, 0.56448598],
       [0.50093458, 0.56448598, 0.97943925],
       [0.56448598, 0.97943925, 0.53831776],
       [0.97943925, 0.53831776, 1.        ],
       [0.53831776, 1.        , 0.42056075],
       [1.        , 0.42056075, 0.79439252],
       [0.42056075, 0.79439252, 0.45794393],
       [0.79439252, 0.45794393, 0.51401869],
       [0.45794393, 0.51401869, 0.77943925],
       [0.51401869, 0.77943925, 0.6635514 ],
       [0.77943925, 0.6635514 , 0.88037383],
       [0.6635514 , 0.88037383, 0.61869159],
       [0.

In [34]:
y_treino

array([0.59439252, 0.73457944, 0.66728972, 0.53457944, 0.54766355,
       0.6728972 , 0.51588785, 0.56635514, 0.50093458, 0.56448598,
       0.97943925, 0.53831776, 1.        , 0.42056075, 0.79439252,
       0.45794393, 0.51401869, 0.77943925, 0.6635514 , 0.88037383,
       0.61869159, 0.94392523, 0.42616822, 0.36635514, 0.94392523,
       0.52897196, 0.27102804, 0.86728972, 0.80186916, 0.51962617,
       0.38691589, 0.00934579, 0.86168224, 0.97570093, 0.        ,
       0.61121495, 0.57757009, 0.70093458, 0.52149533, 0.6317757 ])

In [35]:
# Reshape dos dados para [samples, time steps, features] - Requerimento para o modelo LSTM

X_treino = np.reshape(X_treino, (X_treino.shape[0], X_treino.shape[1], 1))
X_teste = np.reshape(X_teste, (X_teste.shape[0], X_teste.shape[1], 1))

In [39]:
X_treino[:2]

array([[[0.48598131],
        [0.53084112],
        [0.91775701]],

       [[0.53084112],
        [0.91775701],
        [0.59439252]]])

In [38]:
X_teste[:2]

array([[[0.55700935],
        [0.61682243],
        [0.55140187]],

       [[0.61682243],
        [0.55140187],
        [0.58130841]]])

## <font color = "red"> Construção do Modelo LSTM

In [ ]:
# 50 neurônios artificais

modelo = tf.keras.models.Sequential([tf.keras.layers.LSTM(50, input_shape = (look_back, 1)),
                                     tf.keras.layers.Dense(1)])

In [ ]:
# Compila o modelo
# Nesta etapa, aplica o algorítmo de otimização que aplica o back propagation, no caso foi o "adam"
# função de erro 
modelo.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [ ]:
# Treinamento do modelo
# Batch_size = alimenta 1 registro por vez
# verbose mostra na tela o treinmaneot em andamento

modelo.fit(X_treino, y_treino, epochs = 50, batch_size = 1, verbose = 1)

Epoch 1/50
40/40 [==============================] - 2s 3ms/step - loss: 0.2854
Epoch 2/50
40/40 [==============================] - 0s 3ms/step - loss: 0.0759
Epoch 3/50
40/40 [==============================] - 0s 2ms/step - loss: 0.0631
Epoch 4/50
40/40 [==============================] - 0s 2ms/step - loss: 0.0622
Epoch 5/50
40/40 [==============================] - 0s 3ms/step - loss: 0.0615
Epoch 6/50
40/40 [==============================] - 0s 2ms/step - loss: 0.0613
Epoch 7/50
40/40 [==============================] - 0s 2ms/step - loss: 0.0598
Epoch 8/50
40/40 [==============================] - 0s 2ms/step - loss: 0.0598
Epoch 9/50
40/40 [==============================] - 0s 2ms/step - loss: 0.0592
Epoch 10/50
40/40 [==============================] - 0s 2ms/step - loss: 0.0602
Epoch 11/50
40/40 [==============================] - 0s 3ms/step - loss: 0.0581
Epoch 12/50
40/40 [==============================] - 0s 2ms/step - loss: 0.0616
Epoch 13/50
40/40 [==============================

In [43]:
# Fazendo previsões com o modelo

previsão_treino = modelo.predict(X_treino)
previsao_teste = modelo.predict(X_teste)

1/1 [==============================] - 0s 23ms/step


## CALCULANDO ERRO

In [ ]:
# Transformar de volta para a escala orignial e calcular o erro do modelo


